In [3]:
import requests
import os

os.environ["http_proxy"] = ""
os.environ["https_proxy"] = ""

DEFAULT_HOST_IP = "10.112.100.138"


def QnA(request, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:8888/v1/chatqna"

    headers = {"Content-Type": "application/json"}

    response = requests.post(url, headers=headers, json=request)
    return response


def upload_rag_file(rag_file, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:6007/v1/dataprep/ingest"

    with open(rag_file, "rb") as f:
        files = {"files": f}
        response = requests.post(url, files=files)

    return response

def delete_rag_file(rag_file, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:6007/v1/dataprep/delete"
    headers = {"Content-Type": "application/json"}
    payload = {"file_path": rag_file}

    response = requests.post(url, headers=headers, json=payload)
    return response


In [10]:
response=delete_rag_file("results.txt")
print("Status Code:", response.status_code)
print("Response Body:", response.text)

Status Code: 404
Response Body: {"detail":"File not found in db file-keys. Please check file_path."}


In [11]:
# download pdf file
# wget https://raw.githubusercontent.com/opea-project/GenAIComps/v1.1/comps/retrievers/redis/data/nke-10k-2023.pdf

# upload pdf file with dataprep
rag_file = "/home/sdp/lifeng/ai_for_validation/RAG_opea/results.txt"

upload_rag_file_response = upload_rag_file(rag_file)

print("Status Code:", upload_rag_file_response.status_code)
print("Response Body:", upload_rag_file_response.text)

Status Code: 200
Response Body: {"status":200,"message":"Data preparation succeeded"}


In [17]:
request = {
        "messages": "Unit test third_party.torch-xpu-ops.test.xpu.test_linalg_xpu.TestLinalgXPU  test_gemm_bias_offline_tunableop_xpu_bfloat16 returned failed returned: AssertionError: Torch not compiled with CUDA enabled.",
        "stream": False,
        "top_n": 3,       
    }

QnA_response=QnA(request)

print("Status Code:", QnA_response.status_code)
print("Response Body:", QnA_response.text)

if QnA_response.status_code==200:
    result=QnA_response.json()["choices"][0]["message"]["content"]
    answer = result.split("</think>")[-1].strip()
    print(answer)

Status Code: 200
Response Body: {"id":"chatcmpl-bVQSj2LLKXQ2nRVUdXLekB","object":"chat.completion","created":1745389100,"model":"chatqna","choices":[{"index":0,"message":{"role":"assistant","content":"Okay, so I'm trying to figure out why the unit test `third_party.torch-xpu-ops.test.xpu.test_linalg_xpu.TestLinalgXPU.test_gemm_bias_offline_tunableop_xpu_bfloat16` is failing with the error message \"AssertionError: Torch not compiled with CUDA enabled.\" \n\nFirst, I'll look at the error message. It says Torch isn't compiled with CUDA enabled. That makes me think that somewhere in the code, it's trying to use CUDA functions but Torch wasn't built with CUDA support. But wait, the test is for XPU, not CUDA. So why is it complaining about CUDA?\n\nLooking at the search results, I see two issues that might be related. The first is issue 1521, which mentions that PyTorch's Flex Attention fails on XPU with the same error about Torch not being compiled with CUDA enabled. The root cause there i

In [ ]:
QnA_response.json()

In [16]:
result = result.replace("\\n\\n", "\n\n")
print(result)

Okay, so I need to figure out if the failed unit test in PyTorch is a known issue. The test that failed is `third_party.torch-xpu-ops.test.xpu.test_linalg_xpu.TestLinalgXPU.test_gemm_bias_offline_tunableop_xpu_bfloat16`, and the error message is `AssertionError: Torch not compiled with CUDA enabled.` 

First, I'll look at the search results provided. There are two merged results: issue 1521 and issue 302. 

Starting with issue 1521, the description mentions that PyTorch's Flex Attention fails on XPU with the same error message. The root cause is that FlexAttention isn't enabled on XPU yet, and they're targeting it for enablement in torch-2.8. The issue is open, and the assignee is liangan1. 

Looking at issue 302, it's a bit more extensive. It lists several issues related to enabling various CUDA functions on XPU. One of the points mentions that `torch-xpu-ops` version is outdated, leading to skipped test cases. The root cause here is that the `torch-xpu-ops` version used doesn't suppo

In [ ]:
template = """
### You are a helpful, respectful, and honest assistant tasked with determining whether a failed unit test case is a known issue. \
Please refer to the search results from the local knowledge base. All issues retrieved by RAG, regardless of status, if matched, they should be treated as known issues \
If yes, provide the issue id, issue state, issue description, issue owner, issue root cause and solution information. \
If no, return the issue id, issue state and issue owner as N/A and give some insights in issue description, issue root cause and solution. \
Only include information relevant to the question. Do not provide false information if unsure. \
Please generate a valid json for the information collected in English only. \
Please provide details and don't generate unrelated informations not addressed in the prompt. \
Please ensure the generated output is a valid json and without repeated information. \n
"""